# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 89 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          1927 non-null   int64  
 1   match_id                            1927 non-null   object 
 2   goals                               1927 non-null   float64
 3   assists                             1927 non-null   float64
 4   non_penalty_goals                   1927 non-null   float64
 5   penalties_scored                    1927 non-null   float64
 6   penalties_attempted                 1927 non-null   float64
 7   yellow_cards                        1927 non-null   float64
 8   red_cards                           1927 non-null   float64
 9   expected_goals                      1927 non-null   float64
 10  non_penalty_expected_goals          1927 non-null   float64
 11  expected_assisted_goals             1927 no

In [4]:
combined.head()

,Unnamed: 0,match_id,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0,m-00381,0.907776,0.396142,0.738889,0.168887,0.236302,0.093022,-0.048172,0.776569,...,6,4,2,5,11,8,2,1,0,0
1,1,m-00382,0.155008,0.133838,0.123306,0.031702,0.039064,0.131687,-0.000329,0.200098,...,4,1,7,4,11,9,1,1,0,0
2,2,m-00383,-0.039926,-0.056540,-0.137227,0.097301,0.109960,0.066208,0.011733,-0.109844,...,6,9,5,5,9,11,1,2,0,0
3,3,m-00384,0.133878,-0.008924,0.122145,0.011733,0.011733,0.185723,-0.023049,0.212802,...,1,4,2,5,9,8,2,1,0,0
4,4,m-00385,0.401345,0.138483,0.411797,-0.010452,-0.001508,0.071065,-0.018950,0.419927,...,2,5,3,5,11,12,2,2,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
match_columns = [
	"match_id", "competition_id", "home_team_id", "away_team_id", "referee_id",
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]
stats_columns = [
	"goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"penalties_made","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","progressive_passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
player_stats_columns = ["player_id", "minutes_played","ninetys"] + stats_columns
pure_stats_columns = ["minutes_played"] + stats_columns
team_stats_columns = ["team_id"] + stats_columns


## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[stats_columns])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = 69
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
 1/31 [..............................] - ETA: 4s - loss: 53.7012 - accuracy: 0.0000e+00

2024-01-30 12:40:41.360283: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31/31 [==============================] - 0s 510us/step - loss: 27.5590 - accuracy: 0.4004
Epoch 2/500
31/31 [==============================] - 0s 353us/step - loss: 17.0676 - accuracy: 0.4698
Epoch 3/500
31/31 [==============================] - 0s 341us/step - loss: 14.3411 - accuracy: 0.4672
Epoch 4/500
31/31 [==============================] - 0s 351us/step - loss: 11.9706 - accuracy: 0.4711
Epoch 5/500
31/31 [==============================] - 0s 338us/step - loss: 9.7350 - accuracy: 0.4724
Epoch 6/500
31/31 [==============================] - 0s 338us/step - loss: 9.3739 - accuracy: 0.4718
Epoch 7/500
31/31 [==============================] - 0s 356us/step - loss: 8.9230 - accuracy: 0.4711
Epoch 8/500
31/31 [==============================] - 0s 337us/step - loss: 8.5969 - accuracy: 0.4744
Epoch 9/500
31/31 [==============================] - 0s 368us/step - loss: 8.7170 - accuracy: 0.4698
Epoch 10/500
31/31 [==============================] - 0s 336us/step - loss: 8.6140 - accuracy: 0.47

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 392us/step
[ 1.6548785   1.3695203  13.640765   12.319456    4.912462    4.356286
  5.310521    4.9723234  10.668655   11.042262    1.5528402   1.9172255
  0.05920221  0.06480659]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


31/31 [==============================] - 0s 458us/step - loss: 26.3832 - accuracy: 0.4413
Epoch 2/500
31/31 [==============================] - 0s 362us/step - loss: 14.7155 - accuracy: 0.4711
Epoch 3/500
31/31 [==============================] - 0s 357us/step - loss: 12.4299 - accuracy: 0.4685
Epoch 4/500
31/31 [==============================] - 0s 344us/step - loss: 11.0121 - accuracy: 0.4692
Epoch 5/500
31/31 [==============================] - 0s 410us/step - loss: 9.9949 - accuracy: 0.4750
Epoch 6/500
31/31 [==============================] - 0s 363us/step - loss: 9.4122 - accuracy: 0.4750
Epoch 7/500
31/31 [==============================] - 0s 453us/step - loss: 9.0309 - accuracy: 0.4731
Epoch 8/500
31/31 [==============================] - 0s 394us/step - loss: 9.0070 - accuracy: 0.4744
Epoch 9/500
31/31 [==============================] - 0s 744us/step - loss: 8.9548 - accuracy: 0.4750
Epoch 10/500
31/31 [==============================] - 0s 346us/step - loss: 8.6455 - accuracy: 0.47

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 338us/step
[ 1.4635267   1.5482857  12.129898   12.128621    4.1855626   4.258463
  4.5257573   4.211441   10.4492445  10.57149     1.5141679   1.626234
  0.20351619 -0.33540756]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 402us/step - loss: 26.5549 - accuracy: 0.3569
Epoch 2/500
31/31 [==============================] - 0s 368us/step - loss: 14.8562 - accuracy: 0.4672
Epoch 3/500
31/31 [==============================] - 0s 355us/step - loss: 12.3002 - accuracy: 0.4724
Epoch 4/500
31/31 [==============================] - 0s 333us/step - loss: 10.9768 - accuracy: 0.4724
Epoch 5/500
31/31 [==============================] - 0s 324us/step - loss: 10.1528 - accuracy: 0.4692
Epoch 6/500
31/31 [==============================] - 0s 334us/step - loss: 9.5551 - accuracy: 0.4705
Epoch 7/500
31/31 [==============================] - 0s 337us/step - loss: 9.2523 - accuracy: 0.4711
Epoch 8/500
31/31 [==============================] - 0s 409us/step - loss: 9.0775 - accuracy: 0.4698
Epoch 9/500
31/31 [==============================] - 0s 327us/step - loss: 8.8216 - accuracy: 0.4705
Epoch 10/500
31/31 [==============================] - 0s 363us/step - loss: 8.7662 - a

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 349us/step
[ 1.6311245   1.7703956  15.847911   12.37468     5.2077336   4.694983
  5.7127285   4.0689554  12.136313   13.836413    1.5646605   2.1409838
  0.16700652  0.03763108]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 460us/step - loss: 28.2858 - accuracy: 0.4134
Epoch 2/500
31/31 [==============================] - 0s 660us/step - loss: 14.4551 - accuracy: 0.4685
Epoch 3/500
31/31 [==============================] - 0s 581us/step - loss: 11.9383 - accuracy: 0.4711
Epoch 4/500
31/31 [==============================] - 0s 395us/step - loss: 10.7336 - accuracy: 0.4750
Epoch 5/500
31/31 [==============================] - 0s 368us/step - loss: 9.9592 - accuracy: 0.4724
Epoch 6/500
31/31 [==============================] - 0s 464us/step - loss: 9.5025 - accuracy: 0.4711
Epoch 7/500
31/31 [==============================] - 0s 362us/step - loss: 9.2054 - accuracy: 0.4731
Epoch 8/500
31/31 [==============================] - 0s 454us/step - loss: 9.0309 - accuracy: 0.4718
Epoch 9/500
31/31 [==============================] - 0s 362us/step - loss: 8.8027 - accuracy: 0.4757
Epoch 10/500
31/31 [==============================] - 0s 371us/step - loss: 8.7328 - ac

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 347us/step
[ 2.2571533   1.2484113  18.829538    9.622617    6.4676166   3.213012
  6.4905953   3.683463   10.631112   13.32382     1.2604449   2.018496
 -0.19564414 -0.09330227]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 473us/step - loss: 27.1347 - accuracy: 0.3945
Epoch 2/500
31/31 [==============================] - 0s 1ms/step - loss: 13.9388 - accuracy: 0.4685
Epoch 3/500
31/31 [==============================] - 0s 708us/step - loss: 11.7179 - accuracy: 0.4692
Epoch 4/500
31/31 [==============================] - 0s 714us/step - loss: 10.6027 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 415us/step - loss: 9.8988 - accuracy: 0.4705
Epoch 6/500
31/31 [==============================] - 0s 420us/step - loss: 9.4681 - accuracy: 0.4692
Epoch 7/500
31/31 [==============================] - 0s 493us/step - loss: 9.1859 - accuracy: 0.4685
Epoch 8/500
31/31 [==============================] - 0s 433us/step - loss: 8.9585 - accuracy: 0.4737
Epoch 9/500
31/31 [==============================] - 0s 476us/step - loss: 8.7872 - accuracy: 0.4711
Epoch 10/500
31/31 [==============================] - 0s 470us/step - loss: 8.7019 - accu

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 745us/step
[ 1.5255415   1.4172705  13.803936   12.690508    4.5134287   4.8926563
  5.687065    5.0459085  11.330153   12.204935    1.37223     2.0309563
 -0.07077608 -0.13032116]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[stats_columns]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  non_penalty_goals  penalties_scored  \
301  -0.094875 -0.220930          -0.108935          0.014060   
655   1.788608  0.104947           0.194394         -0.115247   
1291  2.641128  0.292058           0.524404          0.105415   
887  -1.201986 -0.302613          -0.124839         -0.038357   
172  -0.174855  0.055731          -0.087689         -0.087167   
...        ...       ...                ...               ...   
132  -0.353935 -0.144334          -0.261522         -0.092413   
479   0.357522  0.089684           0.095154          0.057074   
561   0.382126  0.222171           0.191594          0.139803   
103  -0.438475 -0.195377          -0.396090         -0.042385   
1785  2.166620  0.334845           0.508931          0.099413   

      penalties_attempted  yellow_cards  red_cards  expected_goals  \
301              0.014060      0.317199   0.007532       -0.070804   
655             -0.120927      0.260088   0.009259        0.056842   
1291     

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
n=15
print(X_train)
pca = PCA(n_components = n, random_state=576)
pca.fit(X)
feature_to_pc_map = pd.DataFrame(pca.components_, columns=stats_columns)
feature_to_pc_map.to_csv("../feature_to_15_pcs.csv")

dump(pca, '../prediction_pca.bin')
whole_X_pca = pca.transform(whole_x_train)

pca = PCA(n_components = n, random_state=576)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)


feature_to_pc_map

[[ 0.0031677  -0.66551577 -0.21848639 ...  0.26402707 -0.21099546
   0.84080285]
 [ 0.77367146  0.36703043  0.48064944 ... -0.90530913  0.68286476
   0.02056297]
 [ 1.12242413  0.95989472  1.24128372 ...  0.21158228  1.07150401
   0.92157332]
 ...
 [ 0.19830155  0.73845591  0.47419645 ...  0.43956876  0.4350281
  -0.75557269]
 [-0.13739354 -0.58455231 -0.88034264 ... -0.13458965  0.9829622
   0.02056297]
 [ 0.92831039  1.09546434  1.20561868 ...  0.82841042  1.07906574
  -0.17496408]]


/opt/homebrew/anaconda3/envs/pl-stats/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
0,0.000742,0.000107,0.000126,0.000011,0.000014,0.000180,0.000007,0.000135,0.000124,0.000106,...,0.002635,0.000061,0.000249,0.000798,0.000553,0.000055,0.000024,0.000019,0.000004,0.000001
1,0.003310,0.000432,0.000617,0.000062,0.000075,0.000334,0.000013,0.000655,0.000595,0.000400,...,0.001255,-0.000159,-0.003250,-0.010249,-0.007045,-0.000734,-0.000302,-0.000229,-0.000055,-0.000018
2,0.004623,0.000648,0.001167,0.000053,0.000067,0.000119,0.000005,0.001108,0.001055,0.000620,...,-0.007539,0.000025,0.002575,0.008201,0.005669,0.000545,0.000242,0.000179,0.000045,0.000018
3,0.012079,0.002068,0.002735,0.000449,0.000589,0.002432,0.000033,0.003252,0.002788,0.002198,...,0.025705,0.000104,-0.000951,-0.003529,-0.002580,-0.000551,-0.000192,-0.000159,-0.000006,-0.000026
4,0.014353,0.002036,0.003728,0.000666,0.000896,-0.000929,0.000059,0.003613,0.002910,0.002041,...,-0.071260,-0.000428,0.005889,0.018963,0.013282,0.001974,0.000475,0.000365,0.000071,0.000039
5,0.001594,-0.003195,-0.001510,-0.000459,-0.000585,0.003985,0.000189,-0.000543,-0.000080,-0.002746,...,0.142352,0.001166,-0.001391,-0.003091,-0.001779,-0.000218,0.000231,0.000245,-0.000036,0.000022
6,-0.034167,-0.003478,-0.006656,-0.000571,-0.000644,0.003022,0.000316,-0.006704,-0.006201,-0.002573,...,0.048375,-0.000246,-0.009604,-0.032070,-0.022557,-0.004612,-0.001467,-0.001319,-0.000108,-0.000040
7,0.023980,0.005166,0.004782,0.000819,0.001033,0.001946,0.000030,0.005421,0.004602,0.004799,...,0.100952,0.000392,-0.009461,-0.029945,-0.020579,-0.003916,-0.000768,-0.000626,-0.000166,0.000024
8,0.039591,0.001452,0.011392,0.001563,0.001756,-0.002490,-0.000153,0.012688,0.011300,0.001459,...,0.078385,-0.002122,-0.028237,-0.090180,-0.062402,-0.005697,-0.003281,-0.002815,-0.000393,-0.000073
9,0.068449,0.000357,0.016324,0.002334,0.002614,0.004631,0.000151,0.019519,0.017435,-0.001054,...,-0.049786,0.001774,0.034587,0.102914,0.069006,0.004232,0.003364,0.002524,0.000606,0.000233


The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=stats_columns)
df.index.name = "principle_component_number"
df

,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,-0.114171,-0.128311,-0.116196,-0.066980,-0.068592,-0.131919,-0.078062,-0.118581,-0.120052,-0.133824,...,-0.114326,-0.109473,-0.054349,-0.055403,-0.055586,-0.049778,-0.050294,-0.049469,-0.036578,-0.026929
1,-0.075715,-0.074212,-0.096706,-0.098945,-0.100421,0.005946,0.019051,-0.102108,-0.096502,-0.070409,...,0.041545,0.157093,0.297617,0.300199,0.299421,0.279064,0.294880,0.280418,0.245476,0.169472
2,0.123937,0.090638,0.195662,0.190485,0.192796,-0.093933,-0.079421,0.189504,0.178097,0.079708,...,-0.199990,-0.069102,0.151691,0.153507,0.153764,0.153410,0.154205,0.145504,0.129612,0.094436
3,0.020862,-0.051866,0.056660,0.521509,0.513398,0.045423,-0.002257,0.123345,0.050801,-0.044126,...,0.114465,0.068039,-0.012928,-0.014232,-0.014120,-0.005708,-0.032566,-0.024793,-0.034619,-0.048925
4,0.394274,-0.056730,0.079224,0.003982,-0.019400,0.072308,-0.069869,0.101046,0.115329,-0.047836,...,0.050874,0.001088,-0.029072,-0.029351,-0.031337,-0.026085,0.011984,0.004683,0.043826,-0.007488
5,0.065049,0.117830,-0.177894,0.106389,0.110760,-0.096674,-0.108894,-0.176329,-0.214373,0.138129,...,-0.279605,-0.106282,0.047093,0.042517,0.040150,0.033641,0.012669,0.051467,0.024962,-0.300288
6,-0.026983,-0.090031,0.000993,0.024307,0.035243,0.092312,-0.037146,0.024362,0.021201,-0.053984,...,0.044344,0.007131,0.111855,0.071728,0.052792,-0.029262,0.073182,0.161311,0.175589,-0.822745
7,-0.062073,0.056795,0.033944,-0.039234,-0.054234,-0.040556,-0.862436,0.018233,0.029312,0.032467,...,0.120427,0.018115,-0.017075,-0.015587,-0.013933,-0.008014,-0.022059,0.025128,-0.140254,-0.117219
8,-0.015863,-0.021709,-0.128987,0.147603,0.148072,0.112348,-0.393752,-0.066288,-0.098351,0.005495,...,0.048367,0.029486,0.014670,0.026537,0.031199,-0.018311,0.064567,0.028806,-0.000580,0.369380


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(15, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500


62/62 [==============================] - 0s 402us/step - loss: 42.0098 - accuracy: 0.1317
Epoch 2/500
62/62 [==============================] - 0s 348us/step - loss: 21.0333 - accuracy: 0.4426
Epoch 3/500
62/62 [==============================] - 0s 311us/step - loss: 18.0833 - accuracy: 0.4750
Epoch 4/500
62/62 [==============================] - 0s 298us/step - loss: 17.2143 - accuracy: 0.4802
Epoch 5/500
62/62 [==============================] - 0s 294us/step - loss: 16.9841 - accuracy: 0.4789
Epoch 6/500
62/62 [==============================] - 0s 679us/step - loss: 16.8211 - accuracy: 0.4848
Epoch 7/500
62/62 [==============================] - 0s 705us/step - loss: 16.6820 - accuracy: 0.4789
Epoch 8/500
62/62 [==============================] - 0s 399us/step - loss: 16.6497 - accuracy: 0.4841
Epoch 9/500
62/62 [==============================] - 0s 290us/step - loss: 16.6262 - accuracy: 0.4867
Epoch 10/500
62/62 [==============================] - 0s 291us/step - loss: 16.6488 - accuracy

#### Testing

In [33]:
y_hat = model.predict(X_test)

13/13 [==============================] - 0s 341us/step


In [34]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [35]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) == home_shots+away_shots else 0
	total_shots_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots else 0
	total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if np.floor(home_goals_hat) == home_goals and np.floor(away_goals_hat) == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) == home_fouls + away_fouls else 0
	total_fouls_over += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0

	total_corners += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) == home_corners + away_corners else 0
	total_corners_over += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners else 0
	total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0

	goals_over += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0
	goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

	all_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0
	all_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0

In [36]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.06476683937823834
total_shots_over: 0.5544041450777202
total_shots_under: 0.38082901554404147


total_shots_on_target: 0.10621761658031088
total_shots_on_target_over: 0.44559585492227977
total_shots_on_target_under: 0.4481865284974093


total_booking_points: 0.18134715025906736
total_booking_points_over: 0.17098445595854922
total_booking_points_under: 0.6476683937823834


correct_score: 0.11139896373056994


winner: 0.40932642487046633


total_corners: 0.12953367875647667
total_corners_over: 0.4378238341968912
total_corners_under: 0.4326424870466321


total_fouls: 0.06217616580310881
total_fouls_over: 0.4533678756476684
total_fouls_under: 0.4844559585492228


goals_over: 0.26424870466321243
goals_under: 0.5129533678756477


all_over: 0.0051813471502590676
all_under: 0.06476683937823834


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [37]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(15, activation="relu", input_dim=15))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learn_rate),
		loss="mse",
		metrics=["accuracy"])
	

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [38]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [39]:
# define a grid of the hyperparameter search space
hidden_layer_one = [10,20,30]
learn_rate = [1e-2, 1e-3, 1e-4]
dropout = [0.1, 0.2, 0.3]
batch_size = [16, 32, 64]
epochs = [10, 50, 100, 200]
n_h_layers = [1, 2, 3, 4]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [40]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [41]:
# searcher = GridSearchCV(estimator=model, n_jobs=-2, 
# 	param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

# searchResults = searcher.fit(X_train, y_train)

# bestScore = searchResults.best_score_
# bestParams = searchResults.best_params_
# print(f"[INFO] best score is {bestScore} using {bestParams}")

In [42]:
# tuned_model = searcher.best_params_
# tuned_model

In [43]:
hidden_layer_one = [10]
learn_rate = [1e-4]
dropout = [0.3]
batch_size = [64]
epochs = [10]
n_h_layers = [3]

model = get_mlp_model(hidden_layer_one=10, learn_rate=0.0001, dropout=0.3, n_h_layers=3)
model.fit(X_train, y_train, epochs=500, batch_size=64, )

model.save("../stats_regression_model.h5")
pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/500


25/25 [==============================] - 0s 514us/step - loss: 51.9750 - accuracy: 0.1428
Epoch 2/500
25/25 [==============================] - 0s 458us/step - loss: 51.7569 - accuracy: 0.1434
Epoch 3/500
25/25 [==============================] - 0s 448us/step - loss: 51.4772 - accuracy: 0.1486
Epoch 4/500
25/25 [==============================] - 0s 442us/step - loss: 51.1681 - accuracy: 0.1564
Epoch 5/500
25/25 [==============================] - 0s 877us/step - loss: 50.7983 - accuracy: 0.1564
Epoch 6/500
25/25 [==============================] - 0s 942us/step - loss: 50.4428 - accuracy: 0.1473
Epoch 7/500
25/25 [==============================] - 0s 859us/step - loss: 50.0327 - accuracy: 0.1707
Epoch 8/500
25/25 [==============================] - 0s 438us/step - loss: 49.5862 - accuracy: 0.1772
Epoch 9/500
25/25 [==============================] - 0s 481us/step - loss: 49.0632 - accuracy: 0.2135
Epoch 10/500
25/25 [==============================] - 0s 573us/step - loss: 48.5798 - accuracy

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,pca12,pca13,pca14
0,5.274042,-3.219540,-4.877120,0.963895,-1.281804,0.115342,-1.219804,-1.414106,0.960274,1.264957,0.870691,-0.789563,-0.199695,-0.039636,0.725522
1,2.268956,-1.828847,0.487065,0.469178,-0.386896,0.374516,-0.560729,-0.100360,0.284307,0.243295,0.914146,0.168076,-0.442635,-0.262425,-0.156481
2,0.124269,-0.862653,0.454335,1.566642,0.098724,-0.828177,0.445800,0.751395,0.314801,0.554282,0.564768,-1.090140,0.381376,0.105909,0.091565
3,-1.501177,-3.831168,0.359480,0.514852,-0.012550,1.292645,-0.545749,-0.277123,0.923114,-0.534519,0.167630,0.379044,-0.256961,-0.148490,0.249393
4,-0.869283,-0.516563,-4.176435,-1.067063,-0.397083,1.963987,0.221384,0.031062,1.130409,-0.699811,-0.122993,-0.002760,0.497328,-0.505342,-0.662349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1922,-7.839708,0.378793,-0.373707,-1.371033,-0.889814,0.822076,0.018904,0.471951,-0.796464,0.294342,-0.108072,-0.843714,-0.091597,-0.282067,-0.150806
1923,1.528425,2.650109,-2.559761,1.587689,0.147539,0.165928,0.776278,-0.780382,-0.076042,0.137959,-0.362934,0.094584,0.330405,0.126880,0.633829
1924,0.036830,-2.410749,-0.870812,-1.134889,-1.143512,-0.443795,-0.250571,0.512900,0.722282,0.562799,0.383727,-0.163709,0.121516,-0.105504,0.324525
1925,-5.767255,0.139930,0.637011,-0.751500,-0.679176,0.207154,0.578698,-1.078541,-0.824895,-0.570972,0.169403,-0.126169,-0.295387,0.277792,-0.465338


In [44]:
print(len
	  (X_train))

1541


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?